In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
!pip install fastapi[all] uvicorn nest-asyncio pyngrok diffusers transformers torch torchvision pillow


In [10]:
import nest_asyncio
from pyngrok import ngrok

nest_asyncio.apply()  # Allow asyncio in Colab


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
from pydantic import BaseModel
from PIL import Image
import tempfile
import torch
from diffusers import StableDiffusionPipeline

# === Setup FastAPI ===
app = FastAPI()

# === CORS: Allow all origins (for frontend) ===
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # or restrict to your domain
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# === Load model only once ===
pipe = StableDiffusionPipeline.from_pretrained(
    "Soumyajit94298/patent-backend",
    torch_dtype=torch.float16
).to("cuda")

# === Request schema ===
class PromptRequest(BaseModel):
    prompt: str
    num_images: int = 2
    guidance_scale: float = 7.0

# === Endpoint ===

@app.get("/")
async def root():
    return {"status": "ok"}

@app.get("/generate")
async def root():
    return {"status": "ok"}

@app.post("/generate-image/")
async def generate_image(req: PromptRequest):
    print(f"Received prompt: {req.prompt}")

    # Generate list of images
    images = [pipe(req.prompt, guidance_scale=req.guidance_scale).images[0]
              for _ in range(req.num_images)]

    # Combine them horizontally
    widths, heights = zip(*(img.size for img in images))
    total_width = sum(widths)
    max_height = max(heights)
    combined_img = Image.new("RGB", (total_width, max_height))

    x_offset = 0
    for img in images:
        combined_img.paste(img, (x_offset, 0))
        x_offset += img.size[0]

    # Save to temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp:
        temp_path = tmp.name
        combined_img.save(temp_path)

    print(f"Saved to: {temp_path}")
    return FileResponse(temp_path, media_type="image/png", filename="generated.png")



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
!ngrok config add-authtoken 2vmeblKF8StjbcsnOCU4pJcDXWk_2ELTDDH6bFHhoxXgKn277

In [ ]:
public_url = ngrok.connect(8000)
print("Your API is live at:", public_url)

In [ ]:
import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)